# Chatbot con informacion de agentes de Valorant

Al inicio pense en crear un agente a base de web scrapping que me diera toda la informacion de competitivo de valorant, pero estaba teniendo muchos problemas en el acceso a la informacion de la web para los cuales no tuve tiempo solucionar, por lo cual opte por un bot sencillo que te de la informacion de cualquier agente de valorant basandose en un script (se que es un poco pocho pero no me dio el tiempo :c )

Aunque funciona bien, quisiera en el futuro poder implementar una pagina que de contenidos mas visuales.

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import openai
from google.colab import output

In [2]:
AGENTS_INFO = {
    "Jett": {
        "rol": "Duelista",
        "habilidades": ["Cloudburst", "Updraft", "Tailwind", "Blade Storm"],
        "descripcion": "Agente rápida y letal, con alta movilidad.",
        "nacionalidad": "Corea del Sur",
        "fecha_release": "2 de junio de 2020"
    },
    "Phoenix": {
        "rol": "Duelista",
        "habilidades": ["Blaze", "Curveball", "Hot Hands", "Run It Back"],
        "descripcion": "Puede curarse con sus propias habilidades y es muy ofensivo.",
        "nacionalidad": "Reino Unido",
        "fecha_release": "2 de junio de 2020"
    },
    "Sova": {
        "rol": "Iniciador",
        "habilidades": ["Owl Drone", "Shock Bolt", "Recon Bolt", "Hunter's Fury"],
        "descripcion": "Especialista en información y visión del enemigo.",
        "nacionalidad": "Rusia",
        "fecha_release": "2 de junio de 2020"
    },
    "Sage": {
        "rol": "Centinela",
        "habilidades": ["Barrier Orb", "Slow Orb", "Healing Orb", "Resurrection"],
        "descripcion": "Sanadora del equipo, puede revivir aliados.",
        "nacionalidad": "China",
        "fecha_release": "2 de junio de 2020"
    },
    "Omen": {
        "rol": "Controlador",
        "habilidades": ["Shrouded Step", "Paranoia", "Dark Cover", "From the Shadows"],
        "descripcion": "Especialista en teletransportación y confusión del enemigo.",
        "nacionalidad": "Desconocida",
        "fecha_release": "2 de junio de 2020"
    },
    "Reyna": {
        "rol": "Duelista",
        "habilidades": ["Leer", "Devour", "Dismiss", "Empress"],
        "descripcion": "Una duelista autosuficiente con gran potencial de eliminación.",
        "nacionalidad": "México",
        "fecha_release": "2 de junio de 2020"
    },
    "Killjoy": {
        "rol": "Centinela",
        "habilidades": ["Nanoswarm", "Alarmbot", "Turret", "Lockdown"],
        "descripcion": "Especialista en control de áreas con tecnología avanzada.",
        "nacionalidad": "Alemania",
        "fecha_release": "4 de agosto de 2020"
    },
    "Breach": {
        "rol": "Iniciador",
        "habilidades": ["Aftershock", "Flashpoint", "Fault Line", "Rolling Thunder"],
        "descripcion": "Causa caos con explosiones y cegadoras.",
        "nacionalidad": "Suecia",
        "fecha_release": "2 de junio de 2020"
    },
    "Harbor": {
        "rol": "Controlador",
        "habilidades": ["Cascade", "Cove", "High Tide", "Reckoning"],
        "descripcion": "Manipula el agua para proteger y avanzar.",
        "nacionalidad": "India",
        "fecha_release": "18 de octubre de 2022"
    },
    "Astra": {
        "rol": "Controlador",
        "habilidades": ["Gravity Well", "Nova Pulse", "Nebula", "Astral Form/Cosmic Divide"],
        "descripcion": "Utiliza energía cósmica para controlar el campo de batalla.",
        "nacionalidad": "Ghana",
        "fecha_release": "2 de marzo de 2021"
    },
    "Yoru": {
        "rol": "Duelista",
        "habilidades": ["Fakeout", "Blindside", "Gatecrash", "Dimensional Drift"],
        "descripcion": "Un maestro del engaño y la infiltración.",
        "nacionalidad": "Japón",
        "fecha_release": "12 de enero de 2021"
    },
    "KAY/O": {
        "rol": "Iniciador",
        "habilidades": ["FRAG/ment", "FLASH/drive", "ZERO/point", "NULL/cmd"],
        "descripcion": "Un robot que desactiva las habilidades enemigas.",
        "nacionalidad": "Desconocida",
        "fecha_release": "22 de junio de 2021"
    },
    "Chamber": {
        "rol": "Centinela",
        "habilidades": ["Trademark", "Headhunter", "Rendezvous", "Tour De Force"],
        "descripcion": "Un francotirador con herramientas letales y movilidad.",
        "nacionalidad": "Francia",
        "fecha_release": "16 de noviembre de 2021"
    },
    "Neon": {
        "rol": "Duelista",
        "habilidades": ["Fast Lane", "Relay Bolt", "High Gear", "Overdrive"],
        "descripcion": "Agente filipina que se desplaza a velocidades sorprendentes, descargando ráfagas de radiancia bioeléctrica.",
        "nacionalidad": "Filipinas",
        "fecha_release": "11 de enero de 2022"
    },
    "Fade": {
        "rol": "Iniciador",
        "habilidades": ["Prowler", "Seize", "Haunt", "Nightfall"],
        "descripcion": "Cazadora turca que desata el poder de las pesadillas para capturar secretos enemigos.",
        "nacionalidad": "Turquía",
        "fecha_release": "27 de abril de 2022"
    },
    "Gekko": {
        "rol": "Iniciador",
        "habilidades": ["Dizzy", "Wingman", "Mosh Pit", "Thrash"],
        "descripcion": "Agente estadounidense que lidera un grupo de criaturas para emboscar a los enemigos y plantar o desactivar la spike.",
        "nacionalidad": "Estados Unidos",
        "fecha_release": "7 de marzo de 2023"
    },
    "Deadlock": {
        "rol": "Centinela",
        "habilidades": ["GravNet", "Sonic Sensor", "Barrier Mesh", "Annihilation"],
        "descripcion": "Agente noruega que utiliza tecnología avanzada para detener el avance enemigo.",
        "nacionalidad": "Noruega",
        "fecha_release": "27 de junio de 2023"
    },
    "Vyse": {
        "rol": "Centinela",
        "habilidades": ["Shear", "Arc Rose", "Razorvine", "Steel Garden"],
        "descripcion": "Maestra metálica que utiliza metal líquido para aislar, atrapar y desarmar a sus enemigos, manipulando el campo de batalla a su favor.",
        "nacionalidad": "Desconocida",
        "fecha_release": "28 de agosto de 2024"
    },
    "Tejo": {
        "rol": "Iniciador",
        "habilidades": ["Special Delivery", "Stealth Drone", "Guided Salvo", "Armageddon"],
        "descripcion": "Experto en balística colombiano que utiliza su arsenal para despejar áreas y proporcionar ventajas tácticas a su equipo.",
        "nacionalidad": "Colombia",
        "fecha_release": "8 de enero de 2025"
    },
    "Clove": {
        "rol": "Controlador",
        "habilidades": ["Meddle", "Ruse", "Pick-Me-Up", "Not Dead Yet"],
        "descripcion": "Joven inmortal escocés que crea travesuras para los enemigos tanto en el calor del combate como en el frío de la muerte, manteniendo a los enemigos adivinando incluso desde más allá de la tumba.",
        "nacionalidad": "Escocia",
        "fecha_release": "27 de marzo de 2024"
    }
}



In [3]:
def get_agent_info(agent_name):
    """
    Devuelve información sobre un agente de Valorant.
    """
    agent_name = agent_name.title()
    return AGENTS_INFO.get(agent_name, {"error": "No se encontró información sobre ese agente."})

In [4]:
def chat_with_openai(prompt):
    """
    Chatbot basado en OpenAI para responder preguntas sobre los agentes de Valorant.
    """
    openai.api_key = "TU_API_KEY"
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "system", "content": "Eres un experto en los agentes de Valorant."},
                  {"role": "user", "content": prompt}]
    )
    return response["choices"][0]["message"]["content"]

In [6]:
if __name__ == "__main__":
    output.clear()
    print("✅ Información de los agentes de Valorant cargada.")

    while True:
        agent_name = input("🔹 Escribe el nombre de un agente para obtener información (o escribe 'salir' para terminar): ")
        if agent_name.lower() == "salir":
            print("👋 Saliendo del programa. ¡Hasta luego!")
            break
        agent_info = get_agent_info(agent_name)
        print("📌 Información del agente:", json.dumps(agent_info, indent=4))

        user_question = input("🔹 Pregunta sobre los agentes (o escribe 'salir' para terminar): ")
        if user_question.lower() == "salir":
            print("👋 Saliendo del programa. ¡Hasta luego!")
            break
        response = chat_with_openai(user_question)
        print("🤖 Respuesta del chatbot:", response)



✅ Información de los agentes de Valorant cargada.
🔹 Escribe el nombre de un agente para obtener información (o escribe 'salir' para terminar): salir
👋 Saliendo del programa. ¡Hasta luego!
